## Modelling and Deployment using MLOps

In [7]:
import pandas as pd
import numpy as np
import sys
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from IPython import display
from jiwer import wer

In [8]:
# loading scripts
sys.path.insert(1, '../scripts')
sys.path.append("..")
sys.path.append(".")

from data_cleaning import DataCleaner

DC = DataCleaner("../logs/modelling_notebook.log")

In [9]:
# Read metadata file
train_meta = DC.meta_loader("../data/train_meta.json", "json")
train_meta.head()

,Target,Feature,Output,Duration,n_channel
0,የኤርትራ ወታደሮች ኢትዮጵያውያኑ ን ቀር ባ እንዳ ታይ ና እንዳታ ነጋገር...,../data/train/wav/tr_1025_tr11026.wav,../data/train_new/tr_1025_tr11026.wav,18.956916,2
1,ዘመናዊ ውን የ አምሪ ካን ሂ ዎት ሲ ቀጭ ከርሞ ጰጰሰ ይ ሉናል,../data/train/wav/tr_10603_tr04117.wav,../data/train_new/tr_10603_tr04117.wav,18.956916,2
2,ዛሬ የ በላ ነው ቀይ ወጥ ሁ ርጥ ያለች ና ት,../data/train/wav/tr_10520_tr53128.wav,../data/train_new/tr_10520_tr53128.wav,18.956916,2
3,ዘነበ ች ሰራተኛዋ ን አታ ን ጓጉ ብላ ተቆጣ ቻት,../data/train/wav/tr_10661_tr13126.wav,../data/train_new/tr_10661_tr13126.wav,18.956916,2
4,በጣም ስለሚ ወደው የ መኮንን ወንድም መኮንን ን መኳ ነው የሚለው,../data/train/wav/tr_10244_tr099086.wav,../data/train_new/tr_10244_tr099086.wav,18.956916,2
